<a href="https://colab.research.google.com/github/hajerhajjajifahem24/Repo-2017/blob/master/LSTManalysissentiment!!!!!!!!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re
import string
import nltk 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from google.colab import files
import matplotlib.pyplot as plt

nltk.download('stopwords')
stop_words  = nltk.corpus.stopwords.words('english')
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')



In [ ]:

uploaded = files.upload()

In [ ]:
data = pd.read_csv('googleplaystore_user_reviews.csv')

In [ ]:
data

In [ ]:
data=(data[["Translated_Review","Sentiment"]]).dropna()

In [ ]:
data

In [ ]:
dataN = data[data.Sentiment == "Neutral"]

In [ ]:
dataN

In [ ]:
datap = data[data.Sentiment == "Positive"].value_counts()

In [ ]:
datap

In [ ]:
datapne = data[data.Sentiment == "Negative"].value_counts()

In [ ]:
datapne

**data processing**

In [ ]:
def preprocess_text_text(text):
    text.lower()
   
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text=re.sub(r'[0-9]+', '', text)
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    text_tokens = word_tokenize(text)
    filtered_words = [w for w in text_tokens if not w in stop_words]
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    
    return " ".join(filtered_words)

In [ ]:
data["Translated_Review"]=data.Translated_Review.apply(preprocess_text_text)

In [ ]:
data

In [ ]:
data['Translated_Review'] = data['Translated_Review'].apply(lambda x: x.lower())
# removing special chars
data['Translated_Review'] = data['Translated_Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

data.head()

**`tokenizer data**

---

**bold text**

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Translated_Review'].values)
X = tokenizer.texts_to_sequences(data['Translated_Review'].values)
X = pad_sequences(X)

Y = pd.get_dummies(data['Sentiment']).values


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
X.shape[1]

**model**

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

**fit model**

In [ ]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

**matrix confusion**

In [ ]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

predict

In [ ]:
testtext = ['very bad']
testtext= tokenizer.texts_to_sequences(testtext)
testtext = pad_sequences(testtext, maxlen=29, dtype='int32', value=0)
print(testtext)
sentiment = model.predict(testtext,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
     print("positive") 

upload file reviwes

In [ ]:
uploaded = files.upload()

In [ ]:
!ls

In [ ]:
test = pd.read_csv('reviewsapp.csv')

In [ ]:
test

**data processing**

In [ ]:
test['content'] = test['content'].apply(lambda x: x.lower())
# removing special chars
test['content'] = test['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [ ]:
test['content']=test.content.apply(preprocess_text_text)

In [ ]:
test['content'].unique()

work count

In [ ]:
test['content'][0:10]

**tokenization**

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['content'].values)
X1 = tokenizer.texts_to_sequences(test['content'].values)
X1 = pad_sequences(X)

In [ ]:
len(X1)

In [ ]:

X1.shape[1]

In [ ]:
l=len(X1[0:10])

Predict the reviews

In [ ]:
app_reviews=[]
sentiment=[]
for i in range (l) :
  sentiment= model.predict(X[i] ,batch_size=1,verbose = 2)[0]
  if(np.argmax(sentiment) == 0):
     senti="negative"
      #print("negative")
      
  elif (np.argmax(sentiment) == 1):
        senti="positive"
      #print("positive")

  app_reviews.append(senti)   

In [ ]:
app_reviews

In [ ]:
app_reviews=[]
sentiment = model.predict(X[2] ,batch_size=1,verbose = 2)[0]
sentiment1 = model.predict(X[1] ,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
     senti="negative"
      #print("negative")
elif (np.argmax(sentiment) == 1):
        senti="positive"
      #print("positive")"
if(np.argmax(sentiment1) == 0):
     senti1="negative"
      #print("negative")
elif (np.argmax(sentiment1) == 1):
        senti1="positive"
      #print("positive")"     
app_reviews.append(senti)    
app_reviews.append(senti1)  

In [ ]:
app_reviews

**convert the result to dataframe**

In [ ]:

d = {'sentiment':app_reviews}
app_reviews_data = pd.DataFrame(data=d)
app_reviews_data.to_csv('reviewsapp.csv', index=None, header=True)

In [ ]:
files.download("reviewsapp.csv")

**[bluid corpus of the word ](https:// [link text](https://))**

data prcessing

In [ ]:
STOPWORDS = set(stopwords.words('english'))
corpus=[]
for i in range(0,10000):
    review = re.sub('[^a-zA-Z]', ' ',test['content'][i])
    review = re.sub('[/(){}\[\]\|@!,;]', ' ',test['content'][i])
    #review = re.sub('[^0-9#+_♥️]', ' ',test['content'][i])#Remove bad symbols
    review = review.lower()
    review = review.split()
    stemmer = PorterStemmer()
    review = [stemmer.stem(token) for token in review if token not in STOPWORDS]
    review=' '.join(review)
    corpus.append(review)
corpus

bag of work

In [ ]:
words = []
Sentiment_Polarity=[]
for i in range(0,len(corpus)):
    words = words + (re.findall(r'\w+', corpus[i]))# words cantain all the words in the dataset
    Sentiment_Polarity.append
words